In [ ]:
path = "/kaggle/input/aironly/V_AIRPLANE_001/V_AIRPLANE_001"
path2_imgs = "/kaggle/input/tdl-data/train_data"
path2_csv = "/kaggle/input/tdl-datacsv/finalcsv2.csv"

In [ ]:
!pip uninstall opencv-contrib-python opencv-python --yes

In [ ]:
!pip install opencv-contrib-python

In [ ]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
df = pd.read_csv(path2_csv, index_col='filename')
df.head()
# df.set_index('filename')

In [ ]:
id = 1
idstr = f"V_AIRPLANE_001_{id}"

In [ ]:
img = cv2.imread(f"{path2_imgs}/{idstr}.jpg")
plt.imshow(img)

In [ ]:
for e,filename in enumerate(os.listdir(path2_imgs)):
    if e < 10:
#         print(filename)
        img = cv2.imread(os.path.join(path2_imgs,filename))
        plt.imshow(img)
        imgdata = df.loc[filename]
        x1 = int(imgdata['x_min'])
        y1 = int(imgdata['y_min'])
        x2 = int(imgdata['x_max'])
        y2 = int(imgdata['y_max'])
        cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0), 2)
        plt.figure()
        plt.imshow(img)
        break

In [ ]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
train_images=[]
train_labels=[]


In [ ]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [ ]:
count = 0
for e,filename in enumerate(os.listdir(path2_imgs)):
    try:
        if filename.split('_')[1] == "AIRPLANE":
            imgdata = df.loc[filename]
            x1 = int(imgdata['x_min'])
            y1 = int(imgdata['y_min'])
            x2 = int(imgdata['x_max'])
            y2 = int(imgdata['y_max'])
            #listesh  = list(map(lambda x : math.floor(float(x)) , df.iloc[i+1]['airplane'][1:-1].split(',')))
            image = cv2.imread(os.path.join(path2_imgs,filename))
            gtvalues=[]
            gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
            #print(f"list : {listesh}")
        #         plt.imshow(image)
        #     cv2.rectangle(image,(listesh[0],listesh[1]),(listesh[3]+listesh[0],listesh[2]+listesh[1]),(255,0,0), 2)
        #     plt.figure()
        #     plt.imshow(image)
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast()
            ssresults = ss.process()
            imout = image.copy()
            counter = 0
            falsecounter = 0
            flag = 0
            fflag = 0
            bflag = 0
            print("going to get iou : ")
            for e,result in enumerate(ssresults):
                if e < 2000 and flag == 0:
                    for gtval in gtvalues:
                        x,y,w,h = result
                        iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                        if counter < 30:
                            if iou > 0.70:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append([1,])
                                counter += 1
                        else :
                            fflag =1
                        if falsecounter <30:
                            if iou < 0.3:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append([0,])
                                falsecounter += 1
                        else :
                            bflag = 1
                    if fflag == 1 and bflag == 1:
                        print("inside")
                        flag = 1
            count+=1
            if count > 50:
                print(count)
                break
    #     if i ==1:
    #         break
    except Exception as err:
        if e > 50 :
            break
        print(err)
        pass

In [ ]:
count = 0
for e,filename in enumerate(os.listdir(path2_imgs)):
    try:
        if filename.split('_')[1] == "DRONE":
            imgdata = df.loc[filename]
            x1 = int(imgdata['x_min'])
            y1 = int(imgdata['y_min'])
            x2 = int(imgdata['x_max'])
            y2 = int(imgdata['y_max'])
            #listesh  = list(map(lambda x : math.floor(float(x)) , df.iloc[i+1]['airplane'][1:-1].split(',')))
            image = cv2.imread(os.path.join(path2_imgs,filename))
            gtvalues=[]
            gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
            #print(f"list : {listesh}")
        #         plt.imshow(image)
        #     cv2.rectangle(image,(listesh[0],listesh[1]),(listesh[3]+listesh[0],listesh[2]+listesh[1]),(255,0,0), 2)
        #     plt.figure()
        #     plt.imshow(image)
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast()
            ssresults = ss.process()
            imout = image.copy()
            counter = 0
            falsecounter = 0
            flag = 0
            fflag = 0
            bflag = 0
            print("going to get iou : ")
            for e,result in enumerate(ssresults):
                if e < 2000 and flag == 0:
                    for gtval in gtvalues:
                        x,y,w,h = result
                        iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                        if counter < 30:
                            if iou > 0.70:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append([3,])
                                counter += 1
                        else :
                            fflag =1
                        if falsecounter <30:
                            if iou < 0.3:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append([0,])
                                falsecounter += 1
                        else :
                            bflag = 1
                    if fflag == 1 and bflag == 1:
                        print("inside")
                        flag = 1
            count+=1
            if count > 50:
                print(count)
                break
    #     if i ==1:
    #         break
    except Exception as err:
        if e > 100 :
            break
        print(err)
        pass

In [ ]:
count = 0
for e,filename in enumerate(os.listdir(path2_imgs)):
    try:
        if filename.split('_')[1] == "HELICOPTER":
            imgdata = df.loc[filename]
            x1 = int(imgdata['x_min'])
            y1 = int(imgdata['y_min'])
            x2 = int(imgdata['x_max'])
            y2 = int(imgdata['y_max'])
            #listesh  = list(map(lambda x : math.floor(float(x)) , df.iloc[i+1]['airplane'][1:-1].split(',')))
            image = cv2.imread(os.path.join(path2_imgs,filename))
            gtvalues=[]
            gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
            #print(f"list : {listesh}")
        #         plt.imshow(image)
        #     cv2.rectangle(image,(listesh[0],listesh[1]),(listesh[3]+listesh[0],listesh[2]+listesh[1]),(255,0,0), 2)
        #     plt.figure()
        #     plt.imshow(image)
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast()
            ssresults = ss.process()
            imout = image.copy()
            counter = 0
            falsecounter = 0
            flag = 0
            fflag = 0
            bflag = 0
            print("going to get iou : ")
            for e,result in enumerate(ssresults):
                if e < 2000 and flag == 0:
                    for gtval in gtvalues:
                        x,y,w,h = result
                        iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                        if counter < 30:
                            if iou > 0.55:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append([4,])
                                counter += 1
                        else :
                            fflag =1
                        if falsecounter <30:
                            if iou < 0.45:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append([0,])
                                falsecounter += 1
                        else :
                            bflag = 1
                    if fflag == 1 and bflag == 1:
                        print("inside")
                        flag = 1
            count+=1
            if count > 50:
                print(count)
                break
    #     if i ==1:
    #         break
    except Exception as err:
        if e > 100 :
            break
        print(err)
        pass

In [ ]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

In [ ]:
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

In [ ]:
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

In [ ]:
for layers in (vggmodel.layers)[:15]:
    print(layers)
    layers.trainable = False

In [ ]:
vggmodel.summary()

In [ ]:
X= vggmodel.layers[-2].output

In [ ]:
predictions = Dense(4, activation="softmax")(X)

In [ ]:
model_final = Model(inputs = vggmodel.input, outputs = predictions)

In [ ]:
from keras.optimizers import Adam
opt = Adam(lr=0.0001)

In [ ]:
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])

In [ ]:
model_final.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
# class MyLabelBinarizer(LabelBinarizer):
#     def transform(self, y):
#         Y = super().transform(y)
#         if self.y_type_ == 'binary':
#             return np.hstack((Y, 1-Y))
#         else:
#             return Y
#     def inverse_transform(self, Y, threshold=None):
#         if self.y_type_ == 'binary':
#             return super().inverse_transform(Y[:, 0], threshold)
#         else:
#             return super().inverse_transform(Y, threshold)
x = 0
for i in train_labels:
    if(i[0] != 0):
        x+=1
print(x)

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
Y = mlb.fit_transform(train_labels)
mlb.classes_

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X_new,Y,test_size=0.25)

In [ ]:
trdata = ImageDataGenerator()
traindata = trdata.flow(x=X_train, y=y_train)
tsdata = ImageDataGenerator()
testdata = tsdata.flow(x=X_test, y=y_test)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

In [ ]:
hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 10, epochs= 10, validation_data= testdata, validation_steps=2, callbacks=[checkpoint,early])

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(hist.history["acc"])
# plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss","Validation Loss"])
plt.show()
plt.savefig('chart loss.png')

In [ ]:
# im = X_test[1]
# plt.imshow(im)
# img = np.expand_dims(im, axis=0)
# out= model_final.predict(img)
# if out[0][0] > out[0][1]:
#     print("plane")
# else:
#     print("not plane")

In [ ]:
img = cv2.imread("/kaggle/input/samplepesu/Test_data_images/19.jpeg")
resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
img = np.expand_dims(resized, axis=0)
out= model_final.predict(img)
print(out)

In [ ]:
z=0
for e,i in enumerate(os.listdir(path)):
    if i.startswith("4"):
        z += 1
        img = cv2.imread(os.path.join(path,i))
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = img.copy()
        for e,result in enumerate(ssresults):
            if e < 2000:
                x,y,w,h = result
                timage = imout[y:y+h,x:x+w]
                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)
                out= model_final.predict(img)
                if out[0][0] > 0.65:
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
        plt.figure()
        plt.imshow(imout)

In [ ]:
# import os,cv2,keras
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
model = keras.models.load_model('/kaggle/input/modelp/ieeercnn_vgg16_1.h5')

In [ ]:
model_class = keras.models.load_model('/kaggle/input/classification-model/model_pretrained_classifier.h5')
img.T
img = cv2.imread('/kaggle/input/tdl-data/train_data/V_AIRPLANE_001_1.jpg')
print(model_class.predict(img.T))

In [ ]:
base_path = '/kaggle/input/samplepesu/Test_data_images/'
csv_dict = {}
for i in os.listdir('/kaggle/input/samplepesu/Test_data_images'):
    print(i)
    img = cv2.imread(base_path+i)
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = img.copy()
    #print(ssresults)
    max_confidence = {}
    for e,result in enumerate(ssresults):
        if e < 2000:
            x,y,w,h = result
            timage = imout[y:y+h,x:x+w]
            resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
            img = np.expand_dims(resized, axis=0)
            out= model.predict(img , verbose = 0)
            #print(max_confidence)
            #print(x,y,x+w, y+h)
            #print(out)
    #         for i in out[0]:
    #             if(i>list(max_confidence.keys())[0]):
    #                 max_confidence.clear()
    #                 max_confidence[i] = [imout, (x, y), (x+w, y+h), (255, 0, 0), 1]
            if(out[0][1] > 0.5):
                max_confidence[out[0][1]] = [(x, y), (x+w, y+h), (255, 0, 0), 1]
                #cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)
            elif(out[0][2] > 0.5):
                max_confidence[out[0][2]] = [(x, y), (x+w, y+h), (255, 0, 0), 1]
                #cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)
            elif(out[0][3] > 0.5):
                max_confidence[out[0][3]] = [(x, y), (x+w, y+h), (255, 0, 0), 1]
                #cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)

    #print(max_confidence)
    if(max_confidence == {}):
        print("No element is present in the image ")
        #max_confidence[out[0][0]] = []
        #list_to_csv = [max_confidence[max_confidence_key][0] , max_confidence[max_confidence_key][1]]
        csv_dict[i] = []
    else:
        #print(imout)
        #plt.imshow(imout)
        max_confidence_key = max(list(max_confidence.keys()))
        max_confidence[max_confidence_key]
        list_to_csv = [max_confidence[max_confidence_key][0] , max_confidence[max_confidence_key][1]]
        csv_dict[i] = list_to_csv
        #plt.figure()
#         print("This is the max one ")
#         print(max_confidence[max_confidence_key])
        #cv2.rectangle(imout , max_confidence[max_confidence_key][0] , max_confidence[max_confidence_key][1] , max_confidence[max_confidence_key][2] , max_confidence[max_confidence_key][3])
        # for i in max_confidence:
        #     cv2.rectangle(max_confidence[i][0] , max_confidence[i][1] , max_confidence[i][2] , max_confidence[i][3] , max_confidence[i][4])
        #plt.imshow(imout)


In [ ]:
import csv


In [ ]:
l = [csv_dict]

In [ ]:
field_names = ['a' , 'b']
with open('/kaggle/working/ans.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(l)

In [ ]:
img = cv2.imread("/kaggle/input/samplepesu/Test_data_images/18.jpeg")
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()
imout = img.copy()
#print(ssresults)
for e,result in enumerate(ssresults):
    if e < 2000:
        x,y,w,h = result
        timage = imout[y:y+h,x:x+w]
        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
        img = np.expand_dims(resized, axis=0)
        out= model.predict(img)
        #print(x,y,x+w, y+h)
        #print(out)
        if(out[0][0] != max(out[0])):
            print(out)
        if(out[0][1] > 0.5):
            cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)
        if(out[0][2] > 0.5):
            cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)
        if(out[0][3] > 0.5):
            cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)

plt.figure()
plt.imshow(imout)

In [ ]:
img = cv2.imread("/kaggle/input/tdl-data/train_data/V_AIRPLANE_002_181.jpg")
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()
imout = img.copy()
#print(ssresults)
for e,result in enumerate(ssresults):
    if e < 2000:
        x,y,w,h = result
        timage = imout[y:y+h,x:x+w]
        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
        img = np.expand_dims(resized, axis=0)
        out= model.predict(img)
        #print(x,y,x+w, y+h)
        #print(out)
        if(out[0][0] != max(out[0])):
            print(out)
        if(out[0][1] > 0.3):
            print("im here and i need to box ")
            cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)
        elif(out[0][2] > 0.72):
            cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)
        elif(out[0][3] > 0.6):
            cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1)

plt.figure()
plt.imshow(imout)